In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List

test_fp = r'data/house-prices-advanced-regression-techniques/test.csv'

In [58]:
df = pd.read_csv(test_fp, index_col=0)

df_num = df.select_dtypes(exclude='object')
df_obj = df.select_dtypes(include='object')

print(df.dtypes.value_counts())

object     43
int64      25
float64    11
dtype: int64


In [59]:
df_num.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,52.619342,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,176.753926,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,0.000000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,0.000000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,1526.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [60]:
df_obj.describe()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1455,1459,107,1459,1459,1457,1459,1459,1459,1459,...,1383,1381,1381,1381,1459,3,290,51,1458,1459
unique,5,2,2,4,4,1,5,3,25,9,...,6,3,4,5,3,2,4,3,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,Ex,MnPrv,Shed,WD,Normal
freq,1114,1453,70,934,1311,1457,1081,1396,218,1251,...,853,625,1293,1328,1301,2,172,46,1258,1204


# String Encoding

We got 43 columns of text --> should be modified.
Moreover some of them are:
* 12: filled with score string ('Excellent', 'Good', etc...)
* 5: filled with similar to score string (Good, average, below,...)
* ?: with qualitative string:
    - [brick, stone, wood]
    - [finished,..]
    - [paved, dirt,..]
    - [privacy,...]

We got to modify this string into numerical value. To do so we will:
* find all columns with the string specified above
* convert those columns with numerical value (based on the strings)
* repeat for each column tipe

For the others strings columns we can compare their value to the house price and try to find a first level correlation (for example neighrborhood or garage,...)

In [61]:
def conv_col_str2num(df:pd.DataFrame, str_list:List[str], num_list:List[float]=None)->pd.DataFrame:
    """
    Find columns which contains all strings of the string list.
    Replace those columns strings with the numeric list provided.
    Return the new dataframe of those columns and the columns name

    Args:
        df (pd.DataFrame): this to convert
        str_list (List[str]): Categories to find and to be converted. Expected to be from best to worse
        num_list (List[int], optional): score to replace each of the categories. defaults to a an automatoc score from best to worse.
    
    Returns:
        pd.DataFrame: contains converted categorical values to numeric score
    """

    if num_list is None:
        M = len(str_list) - 1
        num_list = [x/M for x in range(M, -1, -1)]

    x_str = df.isin(str_list).all()
    col = df.columns[x_str]
    df_rep = df[col].replace(str_list, num_list)
    df_new = df.drop(col, axis=1).join(df_rep)

    return df_new

In [62]:
str_lists = [
    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
    ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
    ['Ex', 'Gd', 'TA', 'Fa' 'NA'],
    ['GdPrv', 'MnPrv', 'GdWo', 'MnWw', 'NA'],
    ['Y', 'P', 'N'],
    ['Fin', 'RFn', 'Unf', 'NA'],
    ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal'],
    ['Y', 'N'],
    ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Ubf', 'NA'],
    ['Gd', 'Av', 'Mn', 'No', 'NA'],
    ['Gtl', 'Mod', 'Sev'],
    ['AllPub', 'NoSewr', 'NoSeWa','ELO'],
    ['Lvl', 'Bnk', 'HLS', 'Low'],
    ['Reg', 'IR1', 'IR2', 'IR3'],
]

for str_list in str_lists:
    df_obj = conv_col_str2num(df_obj, str_list)

print(df_obj.dtypes.value_counts())
df_obj

object     38
float64     5
dtype: int64


,MSZoning,Street,Alley,Utilities,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,...,SaleType,SaleCondition,ExterQual,ExterCond,HeatingQC,CentralAir,PavedDrive,LandSlope,LandContour,LotShape
Id,,,,,,,,,,,,,,,,,,,,,
1461,RH,Pave,NaN,AllPub,Inside,NAmes,Feedr,Norm,1Fam,1Story,...,WD,Normal,0.6,0.6,0.6,1.0,1.0,1.0,1.000000,1.000000
1462,RL,Pave,NaN,AllPub,Corner,NAmes,Norm,Norm,1Fam,1Story,...,WD,Normal,0.6,0.6,0.6,1.0,1.0,1.0,1.000000,0.666667
1463,RL,Pave,NaN,AllPub,Inside,Gilbert,Norm,Norm,1Fam,2Story,...,WD,Normal,0.6,0.6,0.8,1.0,1.0,1.0,1.000000,0.666667
1464,RL,Pave,NaN,AllPub,Inside,Gilbert,Norm,Norm,1Fam,2Story,...,WD,Normal,0.6,0.6,1,1.0,1.0,1.0,1.000000,0.666667
1465,RL,Pave,NaN,AllPub,Inside,StoneBr,Norm,Norm,TwnhsE,1Story,...,WD,Normal,0.8,0.6,1,1.0,1.0,1.0,0.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,RM,Pave,NaN,AllPub,Inside,MeadowV,Norm,Norm,Twnhs,2Story,...,WD,Normal,0.6,0.6,0.8,1.0,1.0,1.0,1.000000,1.000000
2916,RM,Pave,NaN,AllPub,Inside,MeadowV,Norm,Norm,TwnhsE,2Story,...,WD,Abnorml,0.6,0.6,0.6,1.0,1.0,1.0,1.000000,1.000000
2917,RL,Pave,NaN,AllPub,Inside,Mitchel,Norm,Norm,1Fam,1Story,...,WD,Abnorml,0.6,0.6,1,1.0,1.0,1.0,1.000000,1.000000
